In [ ]:
pip install git+https://github.com/openai/CLIP.git


In [ ]:
import torch
import clip
import torchvision
from torchvision.datasets import CIFAR10
dataset = CIFAR10(root='data/', download=True)

In [ ]:
device = "cuda" if torch.cuda.is_available() else "cpu"
model, preprocess = clip.load("ViT-B/32", device=device)

num_images=dataset.data.shape[0]

import numpy as np

arr = np.arange(num_images)

np.random.shuffle(arr)

image_labels= np.zeros(num_images, dtype=int)
i=0
for i in range(num_images):
  image_labels[i]=dataset[arr[i]][1]


In [ ]:
image_features=np.zeros([dataset.data.shape[0],512])
with torch.no_grad():
  i=0
  for i in range(num_images):
    img = preprocess(dataset[arr[i]][0]).unsqueeze(0).to(device)
    #img=dataset[i][0]
    #img.save('test.png')
    #image_features[i,:]= model.encode_image(img)
    image_features[i,:]= model.encode_image(img)
    i+=1
    if (i%100==0):
      print('i: '+str(i))

In [ ]:
imax=1000
import pandas as pd
import time
df_subset=pd.DataFrame()
from sklearn.manifold import TSNE
import matplotlib.pyplot as plt


# Let's map 
time_start = time.time()
tsne = TSNE(n_components=2, verbose=1, perplexity=40, n_iter=300)
tsne_results = tsne.fit_transform(image_features[0:imax])
print('t-SNE done! Time elapsed: {} seconds'.format(time.time()-time_start))

In [ ]:
import seaborn as sns

# Create the Plot with t-SNE. We can now clearly distinct different clusters by optical inspection
df_subset['tsne-2d-one'] = tsne_results[:,0]
df_subset['tsne-2d-two'] = tsne_results[:,1]
df_subset['y'] = (image_labels[0:imax])
plt.figure(figsize=(16,10))
sns.scatterplot(
    x="tsne-2d-one", y="tsne-2d-two",
#    color='black',
    hue="y",
    palette=sns.color_palette("hls", 10),
    data=df_subset,
    legend="full",
    alpha=1.0
)